In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card-fraud-data/fraud_data.csv


In [2]:
path = os.path.join(r"/kaggle/input/credit-card-fraud-data/fraud_data.csv")
df = pd.read_csv(path)
print(df.shape)

(14446, 15)


In [3]:
df.head(2)

,trans_date_trans_time,merchant,category,amt,city,state,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud
0,04-01-2019 00:58,"""Stokes, Christiansen and Sipes""",grocery_net,14.37,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,a3806e984cec6ac0096d8184c64ad3a1,65.654142,-164.722603,1
1,04-01-2019 15:06,Predovic Inc,shopping_net,966.11,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,a59185fe1b9ccf21323f581d7477573f,65.468863,-165.473127,1


## DATA CLEANING

In [4]:
# COUNTS OF FRAUD AND NON-FRAUD TRANS
df['is_fraud'].value_counts()

is_fraud
0                         12600
1                          1844
1"2020-12-24 16:56:24"        1
0"2019-01-01 00:00:44"        1
Name: count, dtype: int64

In [5]:
# CORRECTLY FORMAT THE CONTENT OF 'is_fraud' COLUMN
df['is_fraud'] = df.apply(lambda row: int(str(row['is_fraud'])[0]) if str(row['is_fraud'])[0].isdigit() else 0, axis=1)
df['is_fraud'].value_counts()

is_fraud
0    12601
1     1845
Name: count, dtype: int64

In [6]:
df.describe(include='all')

,trans_date_trans_time,merchant,category,amt,city,state,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud
count,14446,14446,14446,14446.000000,14446,14446,14446.000000,14446.000000,1.444600e+04,14446,14446,14446,14446.000000,14446.000000,14446.000000
unique,12126,693,14,NaN,176,13,NaN,NaN,NaN,163,187,14383,NaN,NaN,NaN
top,26-12-2020 23:29,Kilback LLC,grocery_pos,NaN,Phoenix,CA,NaN,NaN,NaN,"""Surveyor, minerals""",17-07-1989,a3806e984cec6ac0096d8184c64ad3a1,NaN,NaN,NaN
freq,5,58,1602,NaN,297,3375,NaN,NaN,NaN,262,197,2,NaN,NaN,NaN
mean,NaN,NaN,NaN,124.430073,NaN,NaN,39.787692,-110.874225,1.065370e+05,NaN,NaN,NaN,39.787991,-110.874892,0.127717
std,NaN,NaN,NaN,231.352587,NaN,NaN,5.317039,12.985813,2.902916e+05,NaN,NaN,NaN,5.360593,12.995596,0.333786
min,NaN,NaN,NaN,1.000000,NaN,NaN,20.027100,-165.672300,4.600000e+01,NaN,NaN,NaN,19.032689,-166.670685,0.000000
25%,NaN,NaN,NaN,12.080000,NaN,NaN,36.715400,-120.415800,4.930000e+02,NaN,NaN,NaN,36.794655,-120.146253,0.000000
50%,NaN,NaN,NaN,51.520000,NaN,NaN,39.666200,-111.098500,1.645000e+03,NaN,NaN,NaN,39.620953,-111.192629,0.000000
75%,NaN,NaN,NaN,101.030000,NaN,NaN,41.940400,-101.136000,3.543900e+04,NaN,NaN,NaN,42.275740,-100.446822,0.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14446 entries, 0 to 14445
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   trans_date_trans_time  14446 non-null  object 
 1   merchant               14446 non-null  object 
 2   category               14446 non-null  object 
 3   amt                    14446 non-null  float64
 4   city                   14446 non-null  object 
 5   state                  14446 non-null  object 
 6   lat                    14446 non-null  float64
 7   long                   14446 non-null  float64
 8   city_pop               14446 non-null  int64  
 9   job                    14446 non-null  object 
 10  dob                    14446 non-null  object 
 11  trans_num              14446 non-null  object 
 12  merch_lat              14446 non-null  float64
 13  merch_long             14446 non-null  float64
 14  is_fraud               14446 non-null  int64  
dtypes:

### Data Descriptions

|Column Name          | Column Description                          |
|---------------------|---------------------------------------------|
| transdatetrans_time | Transaction DateTime                        |
| merchant            | Merchant Name                               |
| category            | Category of Merchant                        |
| amt                 | Amount of Transaction                       |
| city                | City of Credit Card Holder                  |
| state               | State of Credit Card Holder                 |
| lat                 | Latitude Location of Purchase               |
| long                | Longitude Location of Purchase              |
| city_pop            | Credit Card Holder's City Population        |
| job                 | Job of Credit Card Holder                   |
| dob                 | Date of Birth of Credit Card Holder         |
| trans_num           | Transaction Number                          |
| merch_lat           | Latitude Location of Merchant               |
| merch_long          | Longitude Location of Merchant              |
| is_fraud            | Whether Transaction is Fraud (1) or Not (0) |

In [8]:
# CHECK FOR MISSING VALUES
df.isna().sum().sum()

0

In [9]:
# convert 'trans_date_trans_time' and 'dob' columns to datetime
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'], format='%d-%m-%Y %H:%M')
df['dob'] = pd.to_datetime(df['dob'], format='%d-%m-%Y')
# df['trans_date_trans_time'].dt.day   ## USE TO ACCESS
print(df[['trans_date_trans_time', 'dob']].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14446 entries, 0 to 14445
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   trans_date_trans_time  14446 non-null  datetime64[ns]
 1   dob                    14446 non-null  datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 225.8 KB
None


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14446 entries, 0 to 14445
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   trans_date_trans_time  14446 non-null  datetime64[ns]
 1   merchant               14446 non-null  object        
 2   category               14446 non-null  object        
 3   amt                    14446 non-null  float64       
 4   city                   14446 non-null  object        
 5   state                  14446 non-null  object        
 6   lat                    14446 non-null  float64       
 7   long                   14446 non-null  float64       
 8   city_pop               14446 non-null  int64         
 9   job                    14446 non-null  object        
 10  dob                    14446 non-null  datetime64[ns]
 11  trans_num              14446 non-null  object        
 12  merch_lat              14446 non-null  float64       
 13  m

In [11]:
df.columns.values

array(['trans_date_trans_time', 'merchant', 'category', 'amt', 'city',
       'state', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num',
       'merch_lat', 'merch_long', 'is_fraud'], dtype=object)

In [12]:
# CHANGE COLUMNS NAMES WITH MORE APPROPRIATE NAMES
cols = ['trans_date_time', 'merchant_name', 'purchase_category', 'amount', 'purchase_city',
       'purchase_state', 'purchase_city_lat', 'purchase_city_long', 'city_population', 'occupation', 'dateofbirth', 'trans_num',
       'merch_lat', 'merch_long', 'is_fraud']
df.columns = cols
df.head(2)

,trans_date_time,merchant_name,purchase_category,amount,purchase_city,purchase_state,purchase_city_lat,purchase_city_long,city_population,occupation,dateofbirth,trans_num,merch_lat,merch_long,is_fraud
0,2019-01-04 00:58:00,"""Stokes, Christiansen and Sipes""",grocery_net,14.37,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",1939-11-09,a3806e984cec6ac0096d8184c64ad3a1,65.654142,-164.722603,1
1,2019-01-04 15:06:00,Predovic Inc,shopping_net,966.11,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",1939-11-09,a59185fe1b9ccf21323f581d7477573f,65.468863,-165.473127,1


Get year, month, day - (number and name), time - (day and night) from 'trans_date_time'
& age from'dateofbirth

In [13]:
df['trans_year'] = df['trans_date_time'].dt.year
df['trans_month'] = df['trans_date_time'].dt.month
df['trans_day'] = df['trans_date_time'].dt.day
df['trans_hour'] = df['trans_date_time'].dt.hour
df['age'] = df['trans_date_time'].dt.year-df['dateofbirth'].dt.year
df.head()

,trans_date_time,merchant_name,purchase_category,amount,purchase_city,purchase_state,purchase_city_lat,purchase_city_long,city_population,occupation,dateofbirth,trans_num,merch_lat,merch_long,is_fraud,trans_year,trans_month,trans_day,trans_hour,age
0,2019-01-04 00:58:00,"""Stokes, Christiansen and Sipes""",grocery_net,14.37,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",1939-11-09,a3806e984cec6ac0096d8184c64ad3a1,65.654142,-164.722603,1,2019,1,4,0,80
1,2019-01-04 15:06:00,Predovic Inc,shopping_net,966.11,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",1939-11-09,a59185fe1b9ccf21323f581d7477573f,65.468863,-165.473127,1,2019,1,4,15,80
2,2019-01-04 22:37:00,Wisozk and Sons,misc_pos,49.61,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",1939-11-09,86ba3a888b42cd3925881fa34177b4e0,65.347667,-165.914542,1,2019,1,4,22,80
3,2019-01-04 23:06:00,Murray-Smitham,grocery_pos,295.26,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",1939-11-09,3a068fe1d856f0ecedbed33e4b5f4496,64.445035,-166.080207,1,2019,1,4,23,80
4,2019-01-04 23:59:00,Friesen Lt,health_fitness,18.17,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",1939-11-09,891cdd1191028759dc20dc224347a0ff,65.447094,-165.446843,1,2019,1,4,23,80


In [14]:
df.drop(columns=['trans_date_time', 'dateofbirth'], inplace=True)
df.head()

,merchant_name,purchase_category,amount,purchase_city,purchase_state,purchase_city_lat,purchase_city_long,city_population,occupation,trans_num,merch_lat,merch_long,is_fraud,trans_year,trans_month,trans_day,trans_hour,age
0,"""Stokes, Christiansen and Sipes""",grocery_net,14.37,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",a3806e984cec6ac0096d8184c64ad3a1,65.654142,-164.722603,1,2019,1,4,0,80
1,Predovic Inc,shopping_net,966.11,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",a59185fe1b9ccf21323f581d7477573f,65.468863,-165.473127,1,2019,1,4,15,80
2,Wisozk and Sons,misc_pos,49.61,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",86ba3a888b42cd3925881fa34177b4e0,65.347667,-165.914542,1,2019,1,4,22,80
3,Murray-Smitham,grocery_pos,295.26,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",3a068fe1d856f0ecedbed33e4b5f4496,64.445035,-166.080207,1,2019,1,4,23,80
4,Friesen Lt,health_fitness,18.17,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",891cdd1191028759dc20dc224347a0ff,65.447094,-165.446843,1,2019,1,4,23,80


In [15]:
# SETTING ORDER FOR COLUMN NAMES
new_column_order = ['trans_num', 'trans_year', 'trans_month', 'trans_day', 'trans_hour', 
                    'purchase_category', 'purchase_city', 'city_population', 'purchase_state', 
                    'purchase_city_lat', 'purchase_city_long', 'amount', 
                    'age', 'occupation', 'merchant_name', 'merch_lat', 'merch_long', 'is_fraud']
df = df[new_column_order]
print(df.shape)
df.head()

(14446, 18)


,trans_num,trans_year,trans_month,trans_day,trans_hour,purchase_category,purchase_city,city_population,purchase_state,purchase_city_lat,purchase_city_long,amount,age,occupation,merchant_name,merch_lat,merch_long,is_fraud
0,a3806e984cec6ac0096d8184c64ad3a1,2019,1,4,0,grocery_net,Wales,145,AK,64.7556,-165.6723,14.37,80,"""Administrator, education""","""Stokes, Christiansen and Sipes""",65.654142,-164.722603,1
1,a59185fe1b9ccf21323f581d7477573f,2019,1,4,15,shopping_net,Wales,145,AK,64.7556,-165.6723,966.11,80,"""Administrator, education""",Predovic Inc,65.468863,-165.473127,1
2,86ba3a888b42cd3925881fa34177b4e0,2019,1,4,22,misc_pos,Wales,145,AK,64.7556,-165.6723,49.61,80,"""Administrator, education""",Wisozk and Sons,65.347667,-165.914542,1
3,3a068fe1d856f0ecedbed33e4b5f4496,2019,1,4,23,grocery_pos,Wales,145,AK,64.7556,-165.6723,295.26,80,"""Administrator, education""",Murray-Smitham,64.445035,-166.080207,1
4,891cdd1191028759dc20dc224347a0ff,2019,1,4,23,health_fitness,Wales,145,AK,64.7556,-165.6723,18.17,80,"""Administrator, education""",Friesen Lt,65.447094,-165.446843,1


In [16]:
# Drop 'trans_num' as it is not required here - for model 
df.drop(columns=['trans_num'], inplace=True)
df.head()

,trans_year,trans_month,trans_day,trans_hour,purchase_category,purchase_city,city_population,purchase_state,purchase_city_lat,purchase_city_long,amount,age,occupation,merchant_name,merch_lat,merch_long,is_fraud
0,2019,1,4,0,grocery_net,Wales,145,AK,64.7556,-165.6723,14.37,80,"""Administrator, education""","""Stokes, Christiansen and Sipes""",65.654142,-164.722603,1
1,2019,1,4,15,shopping_net,Wales,145,AK,64.7556,-165.6723,966.11,80,"""Administrator, education""",Predovic Inc,65.468863,-165.473127,1
2,2019,1,4,22,misc_pos,Wales,145,AK,64.7556,-165.6723,49.61,80,"""Administrator, education""",Wisozk and Sons,65.347667,-165.914542,1
3,2019,1,4,23,grocery_pos,Wales,145,AK,64.7556,-165.6723,295.26,80,"""Administrator, education""",Murray-Smitham,64.445035,-166.080207,1
4,2019,1,4,23,health_fitness,Wales,145,AK,64.7556,-165.6723,18.17,80,"""Administrator, education""",Friesen Lt,65.447094,-165.446843,1


## Exploratory Data Analysis And Feature Extraction
Now that data is in correct format, lets see how likely a transaction is fraudulent with respect to different features and their categories.

#### EDA And Features Extraction

- [x] Transaction Year
- [x] Transaction Month
- [x] Transaction Day
- [x] Transaction Hour
- [x] Purchase Category
- [x] Purchase City
- [x] Purchase State
- [ ] Purchase City Locations
- [ ] City Population
- [ ] Amount
- [ ] BirthYear
- [ ] Occupation
- [ ] Merchant Name
- [ ] Merchant Location

In [17]:
# First see the categories/unique elements present in each column
for col in df.columns.values:
    print(f'Number of unique values in {col}: {len(list(df[col].unique()))}')

Number of unique values in trans_year: 2
Number of unique values in trans_month: 12
Number of unique values in trans_day: 31
Number of unique values in trans_hour: 24
Number of unique values in purchase_category: 14
Number of unique values in purchase_city: 176
Number of unique values in city_population: 174
Number of unique values in purchase_state: 13
Number of unique values in purchase_city_lat: 183
Number of unique values in purchase_city_long: 183
Number of unique values in amount: 9266
Number of unique values in age: 74
Number of unique values in occupation: 163
Number of unique values in merchant_name: 693
Number of unique values in merch_lat: 14376
Number of unique values in merch_long: 14380
Number of unique values in is_fraud: 2


### 1. Transaction Year Vs Fraud Relation

In [18]:
trans_year_count = df['trans_year'].value_counts()
# trans_year_count.keys()[0]
trans_year_count

trans_year
2019    7643
2020    6803
Name: count, dtype: int64

In [19]:
for year in trans_year_count.keys():
    percent = (df[(df['trans_year']==year) & (df['is_fraud']==1)]['trans_year'].count()/trans_year_count[year])*100
    print(f"Percentage of fraudulent transactions in {year} is: {percent}")

Percentage of fraudulent transactions in 2019 is: 13.620306162501636
Percentage of fraudulent transactions in 2020 is: 11.818315449066588


In [20]:
df_trans_year = df[['trans_year', 'is_fraud']].groupby(by='trans_year').sum().reset_index().sort_values(by='is_fraud', ascending=False)
df_trans_year['total'] = df_trans_year.apply(lambda row: trans_year_count[row['trans_year']], axis=1)
df_trans_year.head()

,trans_year,is_fraud,total
0,2019,1041,7643
1,2020,804,6803


In [21]:
# Initialize figure
fig = go.Figure()

# Add the first bar plot (total transactions) in light blue
fig.add_trace(go.Bar(
    x=df_trans_year['trans_year'], 
    y=df_trans_year['total'], 
    name='Total Transactions',
    marker_color='lightblue'
))

# Percentage value
percent = (df_trans_year['is_fraud']/df_trans_year['total'])*100
txt = percent.apply(lambda x: f'{x: .1f}%')

# Add the second bar plot (fraud transactions) in darker blue, to be overlayed on the first
fig.add_trace(go.Bar(
    x=df_trans_year['trans_year'], 
    y=df_trans_year['is_fraud'], 
    name='Fraud Transactions',
    marker_color='blue',
    text=txt,
    textposition='outside'  # Position the text labels outside the bars
))

# Update layout to overlay the bars
fig.update_layout(
    width=1000,
    height=400,
    barmode='overlay',  # Overlay the bars on top of each other
    title='Total vs. Fraudulent Transactions by Year',
    xaxis_title='Transaction Year',
    yaxis_title='Count of Transactions',
    xaxis=dict(
        tickmode='linear',  # Set x-ticks at regular intervals
        tick0=1,  # Starting tick value
        dtick=1,  # Interval between ticks
    ),
)
fig.show()

Observations -
1. There is no big difference in fraud percentage for two years
2. We might as well drop this column - as it is not giving much of the distinction

In [22]:
df.drop(columns=['trans_year'], inplace=True)
df.head()

,trans_month,trans_day,trans_hour,purchase_category,purchase_city,city_population,purchase_state,purchase_city_lat,purchase_city_long,amount,age,occupation,merchant_name,merch_lat,merch_long,is_fraud
0,1,4,0,grocery_net,Wales,145,AK,64.7556,-165.6723,14.37,80,"""Administrator, education""","""Stokes, Christiansen and Sipes""",65.654142,-164.722603,1
1,1,4,15,shopping_net,Wales,145,AK,64.7556,-165.6723,966.11,80,"""Administrator, education""",Predovic Inc,65.468863,-165.473127,1
2,1,4,22,misc_pos,Wales,145,AK,64.7556,-165.6723,49.61,80,"""Administrator, education""",Wisozk and Sons,65.347667,-165.914542,1
3,1,4,23,grocery_pos,Wales,145,AK,64.7556,-165.6723,295.26,80,"""Administrator, education""",Murray-Smitham,64.445035,-166.080207,1
4,1,4,23,health_fitness,Wales,145,AK,64.7556,-165.6723,18.17,80,"""Administrator, education""",Friesen Lt,65.447094,-165.446843,1


### 2. Transaction month Vs Fraud Relation

In [23]:
trans_month_counts = df['trans_month'].value_counts()
trans_month_counts

trans_month
1     6806
12    6161
3      220
9      197
2      169
5      167
8      156
10     150
7      125
4      120
6       92
11      83
Name: count, dtype: int64

In [24]:
df_trans_month = df[['trans_month', 'is_fraud']].groupby(by='trans_month').sum().reset_index().sort_values(by='is_fraud', ascending=False)
df_trans_month['total'] = df_trans_month.apply(lambda row: trans_month_counts[row['trans_month']], axis=1)
df_trans_month.head()

,trans_month,is_fraud,total
2,3,220,220
0,1,204,6806
8,9,197,197
1,2,169,169
4,5,167,167


In [25]:
# Initialize figure
fig = go.Figure()

# Add the first bar plot (total transactions) in light blue
fig.add_trace(go.Bar(
    x=df_trans_month['trans_month'], 
    y=df_trans_month['total'], 
    name='Total Transactions',
    marker_color='lightblue'
))

# Percentage value
percent = (df_trans_month['is_fraud']/df_trans_month['total'])*100
txt = percent.apply(lambda x: f'{x: .1f}%')

# Add the second bar plot (fraud transactions) in darker blue, to be overlayed on the first
fig.add_trace(go.Bar(
    x=df_trans_month['trans_month'], 
    y=df_trans_month['is_fraud'], 
    name='Fraud Transactions',
    marker_color='blue',
    text=txt,
    textposition='outside'  # Position the text labels outside the bars
))

# Update layout to overlay the bars
fig.update_layout(
    width=1000,
    height=400,
    barmode='overlay',  # Overlay the bars on top of each other
    title='Total vs. Fraudulent Transactions by Month',
    xaxis_title='Transaction Month',
    yaxis_title='Count of Transactions',
    xaxis=dict(
        tickmode='linear',  # Set x-ticks at regular intervals
        tick0=1,  # Starting tick value
        dtick=1,  # Interval between ticks
        range=[0, len(df_trans_month['trans_month'])+1]  # Set x-axis limits (xlim)
    ),
    yaxis=dict(
#         range=[0, 1000]  # Set y-axis limits (xlim)
    )
)

fig.show()


Observation -
1. The bar plot clearly shows that the transactions occured in months excluding [1st and 12th] are all fraud
2. This is very interesting and might suggest that data is not real - because it is highly unlikely that fruad are only happening in Jan and Dec months
3. However, the count of fraud transactions are in similar range for all months - thus 'trans_month' does not provide distinct diff btw fraud and non-fraud transactions - therefore we will drop this column as well

In [26]:
df.drop(columns=['trans_month'], inplace=True)
df.head()

,trans_day,trans_hour,purchase_category,purchase_city,city_population,purchase_state,purchase_city_lat,purchase_city_long,amount,age,occupation,merchant_name,merch_lat,merch_long,is_fraud
0,4,0,grocery_net,Wales,145,AK,64.7556,-165.6723,14.37,80,"""Administrator, education""","""Stokes, Christiansen and Sipes""",65.654142,-164.722603,1
1,4,15,shopping_net,Wales,145,AK,64.7556,-165.6723,966.11,80,"""Administrator, education""",Predovic Inc,65.468863,-165.473127,1
2,4,22,misc_pos,Wales,145,AK,64.7556,-165.6723,49.61,80,"""Administrator, education""",Wisozk and Sons,65.347667,-165.914542,1
3,4,23,grocery_pos,Wales,145,AK,64.7556,-165.6723,295.26,80,"""Administrator, education""",Murray-Smitham,64.445035,-166.080207,1
4,4,23,health_fitness,Wales,145,AK,64.7556,-165.6723,18.17,80,"""Administrator, education""",Friesen Lt,65.447094,-165.446843,1


### 3. Transaction Day Vs Fraud Relation

In [27]:
trans_day_counts = df['trans_day'].value_counts()
# len(trans_day_counts)
trans_day_counts.head()

trans_day
29    1198
28    1160
27    1053
26     757
31     722
Name: count, dtype: int64

In [28]:
df_trans_day = df[['trans_day', 'is_fraud']].groupby(by='trans_day').sum().reset_index().sort_values(by='is_fraud', ascending=False)
df_trans_day['total'] = df_trans_day.apply(lambda row: trans_day_counts[row['trans_day']], axis=1)
df_trans_day.head()

,trans_day,is_fraud,total
13,14,122,516
12,13,93,465
29,30,80,602
10,11,80,355
14,15,76,476


In [29]:
# Initialize figure
fig = go.Figure()

# Add the first bar plot (total transactions) in light blue
fig.add_trace(go.Bar(
    x=df_trans_day['trans_day'], 
    y=df_trans_day['total'], 
    name='Total Transactions',
    marker_color='lightblue'
))

# Percentage value
percent = (df_trans_day['is_fraud']/df_trans_day['total'])*100
txt = percent.apply(lambda x: f'{x: .1f}%')

# Add the second bar plot (fraud transactions) in darker blue, to be overlayed on the first
fig.add_trace(go.Bar(
    x=df_trans_day['trans_day'], 
    y=df_trans_day['is_fraud'], 
    name='Fraud Transactions',
    marker_color='blue',
    text=txt,
    textposition='outside'  # Position the text labels outside the bars
))

# Update layout to overlay the bars
fig.update_layout(
    width=1000,
    height=400,
    barmode='overlay',  # Overlay the bars on top of each other
    title='Total vs. Fraudulent Transactions by Day',
    xaxis_title='Transaction Day',
    yaxis_title='Count of Transactions',
    xaxis=dict(
        tickmode='linear',  # Set x-ticks at regular intervals
        tick0=1,  # Starting tick value
        dtick=1,  # Interval between ticks
        range=[0, len(df_trans_day['trans_day'])+1]  # Set x-axis limits (xlim)
    ),
    yaxis=dict(
#         range=[0, 150]  # Set y-axis limits (xlim)
    )   
)

fig.show()

Observations - 
1. The fraud percentage varies across day (numbers) - however the count of fraud transaction is similar for all days
2. Therefore we will drop 'trans_day' column as well

In [30]:
df.drop(columns=['trans_day'], inplace=True)
df.head()

,trans_hour,purchase_category,purchase_city,city_population,purchase_state,purchase_city_lat,purchase_city_long,amount,age,occupation,merchant_name,merch_lat,merch_long,is_fraud
0,0,grocery_net,Wales,145,AK,64.7556,-165.6723,14.37,80,"""Administrator, education""","""Stokes, Christiansen and Sipes""",65.654142,-164.722603,1
1,15,shopping_net,Wales,145,AK,64.7556,-165.6723,966.11,80,"""Administrator, education""",Predovic Inc,65.468863,-165.473127,1
2,22,misc_pos,Wales,145,AK,64.7556,-165.6723,49.61,80,"""Administrator, education""",Wisozk and Sons,65.347667,-165.914542,1
3,23,grocery_pos,Wales,145,AK,64.7556,-165.6723,295.26,80,"""Administrator, education""",Murray-Smitham,64.445035,-166.080207,1
4,23,health_fitness,Wales,145,AK,64.7556,-165.6723,18.17,80,"""Administrator, education""",Friesen Lt,65.447094,-165.446843,1


### 4. Transaction Hour Vs Fraud Relation

In [31]:
trans_hour_counts = df['trans_hour'].value_counts()
trans_hour_counts.head()

trans_hour
23    1127
22    1120
20     702
21     692
15     671
Name: count, dtype: int64

In [32]:
df_trans_hour = df[['trans_hour', 'is_fraud']].groupby(by='trans_hour').sum().reset_index().sort_values(by='is_fraud', ascending=False)
df_trans_hour['total'] = df_trans_hour.apply(lambda row: trans_hour_counts[row['trans_hour']], axis=1)
df_trans_hour.head()

,trans_hour,is_fraud,total
23,23,477,1127
22,22,468,1120
1,1,174,613
0,0,174,587
3,3,147,564


In [33]:
# Initialize figure
fig = go.Figure()

# Add the first bar plot (total transactions) in light blue
fig.add_trace(go.Bar(
    x=df_trans_hour['trans_hour'], 
    y=df_trans_hour['total'], 
    name='Total Transactions',
    marker_color='lightblue'
))

# Percentage value
percent = (df_trans_hour['is_fraud']/df_trans_hour['total'])*100
txt = percent.apply(lambda x: f'{x: .1f}%')

# Add the second bar plot (fraud transactions) in darker blue, to be overlayed on the first
fig.add_trace(go.Bar(
    x=df_trans_hour['trans_hour'], 
    y=df_trans_hour['is_fraud'], 
    name='Fraud Transactions',
    marker_color='blue',
    text=txt,
    textposition='outside'  # Position the text labels outside the bars
))

# Update layout to overlay the bars
fig.update_layout(
    width=1000,
    height=400,
    barmode='overlay',  # Overlay the bars on top of each other
    title='Total vs. Fraudulent Transactions by Hour',
    xaxis_title='Transaction Hour',
    yaxis_title='Count of Transactions',
    xaxis=dict(
        tickmode='linear',  # Set x-ticks at regular intervals
        tick0=1,  # Starting tick value
        dtick=1,  # Interval between ticks
        range=[-1, len(df_trans_hour['trans_hour'])]  # Set x-axis limits (xlim)
    ),
    yaxis=dict(
#         range=[0, 150]  # Set y-axis limits (xlim)
    )   
)
fig.show()

Observations - 
1. The high fraud percentage during night hours indicates that most fraudulent transactions occur at this time
2. This clear distinction is very useful features for our model training - so keep the column

### 5. Purchase Category Vs Fraud Relation

In [34]:
purchase_category_count = df['purchase_category'].value_counts()
purchase_category_count

purchase_category
grocery_pos       1602
gas_transport     1430
shopping_net      1408
shopping_pos      1354
home              1304
kids_pets         1141
personal_care      990
entertainment      953
health_fitness     891
food_dining        870
misc_pos           823
misc_net           821
grocery_net        474
travel             385
Name: count, dtype: int64

In [35]:
df_purchase_category = df[['purchase_category', 'is_fraud']].groupby(by='purchase_category').sum().reset_index().sort_values(by='is_fraud', ascending=False)
df_purchase_category['total'] = df_purchase_category.apply(lambda row: purchase_category_count[row['purchase_category']], axis=1)
df_purchase_category.head()

,purchase_category,is_fraud,total
4,grocery_pos,444,1602
11,shopping_net,396,1408
8,misc_net,223,821
12,shopping_pos,194,1354
2,gas_transport,159,1430


In [36]:
# Initialize figure
fig = go.Figure()

# Add the first bar plot (total transactions) in light blue
fig.add_trace(go.Bar(
    x=df_purchase_category['purchase_category'], 
    y=df_purchase_category['total'], 
    name='Total Transactions',
    marker_color='lightblue'
))

# Percentage value
percent = (df_purchase_category['is_fraud']/df_purchase_category['total'])*100
txt = percent.apply(lambda x: f'{x: .1f}%')

# Add the second bar plot (fraud transactions) in darker blue, to be overlayed on the first
fig.add_trace(go.Bar(
    x=df_purchase_category['purchase_category'], 
    y=df_purchase_category['is_fraud'], 
    name='Fraud Transactions',
    marker_color='blue',
    text=txt,
    textposition='outside'  # Position the text labels outside the bars
))

# Update layout to overlay the bars
fig.update_layout(
    width=1000,
    height=400,
    barmode='overlay',  # Overlay the bars on top of each other
    title='Total vs. Fraudulent Transactions by Purchase Category',
    xaxis_title='Purchase Category',
    yaxis_title='Count of Transactions',
    xaxis=dict(
        tickmode='linear',  # Set x-ticks at regular intervals
        tick0=1,  # Starting tick value
        dtick=1,  # Interval between ticks
        range=[-1, len(df_purchase_category['purchase_category'])]  # Set x-axis limits (xlim)
    ),
    yaxis=dict(
#         range=[0, 150]  # Set y-axis limits (xlim)
    )   
)
fig.show()

Observations - 
1. 'purchase_category' - useful feature - since there is diff btw the fraud percentages 

### 6. Purchase City Vs Fraud Relation

In [37]:
purchase_city_count = df['purchase_city'].value_counts()
print(f"Total Number of cities: {len(purchase_city_count)}")
purchase_city_count.head()

Total Number of cities: 176


purchase_city
Phoenix          297
Centerview       197
Orient           192
Sutherland       187
Fort Washakie    187
Name: count, dtype: int64

In [38]:
df_purchase_city = df[['purchase_city', 'is_fraud']].groupby(by='purchase_city').sum().reset_index().sort_values(by='is_fraud', ascending=False)
df_purchase_city['total'] = df_purchase_city.apply(lambda row: purchase_city_count[row['purchase_city']], axis=1)

# To show the percentage diff btw the sorted df - we will select the first 15 and last 15 rows
df_purchase_city = pd.concat([df_purchase_city.head(10), df_purchase_city.tail(10)], ignore_index=True)
# df_purchase_city = df_purchase_city.head(30)
# df_purchase_city = df_purchase_city.tail(30)
df_purchase_city.head()

,purchase_city,is_fraud,total
0,Wales,30,111
1,San Jose,27,148
2,Albuquerque,24,81
3,Thompson,24,178
4,Aurora,23,47


In [39]:
# Initialize figure
fig = go.Figure()

# Add the first bar plot (total transactions) in light blue
fig.add_trace(go.Bar(
    x=df_purchase_city['purchase_city'], 
    y=df_purchase_city['total'], 
    name='Total Transactions',
    marker_color='lightblue'
))

# Percentage value
percent = (df_purchase_city['is_fraud']/df_purchase_city['total'])*100
txt = percent.apply(lambda x: f'{x: .1f}%')

# Add the second bar plot (fraud transactions) in darker blue, to be overlayed on the first
fig.add_trace(go.Bar(
    x=df_purchase_city['purchase_city'], 
    y=df_purchase_city['is_fraud'], 
    name='Fraud Transactions',
    marker_color='blue',
    text=txt,
    textposition='outside'  # Position the text labels outside the bars
))

# Update layout to overlay the bars
fig.update_layout(
    width=1000,
    height=400,
    barmode='overlay',  # Overlay the bars on top of each other
    title='Total vs. Fraudulent Transactions by Purchase City (Top 10 and Bottom 10)',
    xaxis_title='Purchase City',
    yaxis_title='Count of Transactions',
    xaxis=dict(
        tickmode='linear',  # Set x-ticks at regular intervals
        tick0=1,  # Starting tick value
        dtick=1,  # Interval between ticks
        range=[-1, len(df_purchase_city['purchase_city'])]  # Set x-axis limits (xlim)
    ),
    yaxis=dict(
#         range=[0, 150]  # Set y-axis limits (xlim)
    )   
)
fig.show()

#### Visualizing city coordinates with fraud count

In [40]:
maps = df[df['is_fraud'] == 1][['purchase_city', 'purchase_city_lat', 'purchase_city_long']].drop_duplicates().sort_values('purchase_city').reset_index(drop=True)

count = df[df['is_fraud'] == 1]['purchase_city'].value_counts().rename_axis('purchase_city').reset_index(name='count')
maps = maps.merge(count, on='purchase_city', how='left').sort_values('count')
maps.head()

,purchase_city,purchase_city_lat,purchase_city_long,count
85,Littleton,39.5994,-105.0044,2
84,Laramie,41.4247,-105.4781,2
115,Odessa,38.9829,-93.9757,3
111,Newhall,34.3795,-118.5230,3
169,Vancouver,45.6892,-122.6616,3


In [41]:
fig = px.density_mapbox(
    maps, lat='purchase_city_lat', lon='purchase_city_long', z='count',
    color_continuous_scale='Plasma', radius=10,
    title='Citywise fraud count Density map',
    center={"lat": 39.8283, "lon": -98.5795}, zoom=3,
    mapbox_style="open-street-map", hover_name='purchase_city',
    height=500, width=1000
)
fig.show()

We will drop the (city_lat, city_long) columns as city itself represent it

In [42]:
df.drop(columns=['purchase_city'], inplace=True)
df.head(2)

,trans_hour,purchase_category,city_population,purchase_state,purchase_city_lat,purchase_city_long,amount,age,occupation,merchant_name,merch_lat,merch_long,is_fraud
0,0,grocery_net,145,AK,64.7556,-165.6723,14.37,80,"""Administrator, education""","""Stokes, Christiansen and Sipes""",65.654142,-164.722603,1
1,15,shopping_net,145,AK,64.7556,-165.6723,966.11,80,"""Administrator, education""",Predovic Inc,65.468863,-165.473127,1


### 7. City Population Vs Fraud Relation

In [43]:
fig = px.histogram(df, x="city_population", color='is_fraud', nbins=100)

fig.update_layout(
    width=1000,
    height=400,
    title='Distribution of Population of Cities w.r.t. Fraud Count',
    xaxis_title='City Population (Bins)',
    yaxis_title='Population Count',
    yaxis=dict(
#         range=[0, 500]
    )
)
fig.show()

Observations - 
1. The distribution of population of cities shows concentration of city population within half million.
2. Majority (Almost all) frauds are occured for city population within 49900
3. I'm not sure how this will help in model training/prediction - but we will take this column

### 8. Purchase State Vs Fraud Relation

In [44]:
purchase_state_count = df['purchase_state'].value_counts()
print(len(purchase_state_count))
purchase_state_count.head()

13


purchase_state
CA    3375
MO    2329
NE    1460
OR    1211
WA    1150
Name: count, dtype: int64

In [45]:
df_purchase_state = df[['purchase_state', 'is_fraud']].groupby(by='purchase_state').sum().reset_index().sort_values(by='is_fraud', ascending=False)
df_purchase_state['total'] = df_purchase_state.apply(lambda row: purchase_state_count[row['purchase_state']], axis=1)
df_purchase_state.head()

,purchase_state,is_fraud,total
2,CA,411,3375
6,MO,267,2329
7,NE,238,1460
9,OR,197,1211
11,WA,126,1150


In [46]:
# Initialize figure
fig = go.Figure()

# Add the first bar plot (total transactions) in light blue
fig.add_trace(go.Bar(
    x=df_purchase_state['purchase_state'], 
    y=df_purchase_state['total'], 
    name='Total Transactions',
    marker_color='lightblue'
))

# Percentage value
percent = (df_purchase_state['is_fraud']/df_purchase_state['total'])*100
txt = percent.apply(lambda x: f'{x: .1f}%')

# Add the second bar plot (fraud transactions) in darker blue, to be overlayed on the first
fig.add_trace(go.Bar(
    x=df_purchase_state['purchase_state'], 
    y=df_purchase_state['is_fraud'], 
    name='Fraud Transactions',
    marker_color='blue',
    text=txt,
    textposition='outside'  # Position the text labels outside the bars
))

# Update layout to overlay the bars
fig.update_layout(
    width=1000,
    height=400,
    barmode='overlay',  # Overlay the bars on top of each other
    title='Total vs. Fraudulent Transactions by Purchase State',
    xaxis_title='Purchase State',
    yaxis_title='Count of Transactions',
    xaxis=dict(
        tickmode='linear',  # Set x-ticks at regular intervals
        tick0=1,  # Starting tick value
        dtick=1,  # Interval between ticks
        range=[-1, len(df_purchase_state['purchase_state'])]  # Set x-axis limits (xlim)
    ),
    yaxis=dict(
#         range=[0, 150]  # Set y-axis limits (xlim)
    )   
)
fig.show()

Observations - 
1. Fraud percentage varies across states (9-16%) - However, this provides less distinction btw states w.r.t. fraud percentage
2. Therefore we will drop this column as well

In [47]:
df.drop(columns=['purchase_state'], inplace=True)
df.head(2)

,trans_hour,purchase_category,city_population,purchase_city_lat,purchase_city_long,amount,age,occupation,merchant_name,merch_lat,merch_long,is_fraud
0,0,grocery_net,145,64.7556,-165.6723,14.37,80,"""Administrator, education""","""Stokes, Christiansen and Sipes""",65.654142,-164.722603,1
1,15,shopping_net,145,64.7556,-165.6723,966.11,80,"""Administrator, education""",Predovic Inc,65.468863,-165.473127,1


### 9. Transaction Amount Vs Fraud Relation

In [48]:
fig = px.histogram(df, x="amount", color='is_fraud', nbins=30)

fig.update_layout(
    width=1000,
    height=400,
    title='Distribution of Transaction Amount w.r.t. Fraud Count',
    xaxis_title='Amount (Bins)',
    yaxis_title='Amount Count',
    yaxis=dict(
#         range=[0, 500]
    )
)
fig.show()

Observations - 
1. Fraud amounts are within 0 to 1000

### 10 Age Vs Fraud Relation

In [49]:
fig = px.histogram(df, x="age", color='is_fraud', nbins=10)

fig.update_layout(
    width=1000,
    height=400,
    title='Distribution of Age w.r.t. Fraud Count',
    xaxis_title='Age (Bins)',
    yaxis_title='Age Count',
    yaxis=dict(
#         range=[0, 500]
    )
)
fig.show()

Observations -
1. The distribution of age values shows that - fraud is occured for age groups from 20 to 90 with most for (50-60)

### 11. Occupation VS Fraud Relation

In [50]:
occupation_counts = df['occupation'].value_counts()
occupation_counts.head()

occupation
"Surveyor, minerals"          262
"Surveyor, land/geomatics"    240
Land/geomatics surveyor       225
Insurance broker              209
Electronics engineer          197
Name: count, dtype: int64

In [51]:
df_occupation = df[['occupation', 'is_fraud']].groupby(by='occupation').sum().reset_index().sort_values(by='is_fraud', ascending=False)
df_occupation['total'] = df_occupation.apply(lambda row: occupation_counts[row['occupation']], axis=1)
df_occupation = pd.concat([df_occupation.head(10), df_occupation.tail(10)], ignore_index=True)
df_occupation.head()

,occupation,is_fraud,total
0,"""Surveyor, minerals""",37,262
1,"""Administrator, education""",30,111
2,Colour technologist,29,111
3,Systems analyst,28,192
4,"""Surveyor, land/geomatics""",24,240


In [52]:
# Initialize figure
fig = go.Figure()

# Add the first bar plot (total transactions) in light blue
fig.add_trace(go.Bar(
    x=df_occupation['occupation'], 
    y=df_occupation['total'], 
    name='Total Transactions',
    marker_color='lightblue'
))

# Percentage value
percent = (df_occupation['is_fraud']/df_occupation['total'])*100
txt = percent.apply(lambda x: f'{x: .1f}%')

# Add the second bar plot (fraud transactions) in darker blue, to be overlayed on the first
fig.add_trace(go.Bar(
    x=df_occupation['occupation'], 
    y=df_occupation['is_fraud'], 
    name='Fraud Transactions',
    marker_color='blue',
    text=txt,
    textposition='outside'  # Position the text labels outside the bars
))

# Update layout to overlay the bars
fig.update_layout(
    width=1000,
    height=400,
    barmode='overlay',  # Overlay the bars on top of each other
    title='Total vs. Fraudulent Transactions by Occupation (Top 10 & Bottom 10)',
    xaxis_title='Occupation',
    yaxis_title='Count of Transactions',
    xaxis=dict(
        tickmode='linear',  # Set x-ticks at regular intervals
        tick0=1,  # Starting tick value
        dtick=1,  # Interval between ticks
        range=[-1, len(df_occupation['occupation'])]  # Set x-axis limits (xlim)
    ),
    yaxis=dict(
#         range=[0, 150]  # Set y-axis limits (xlim)
    )   
)
fig.show()

### 12. Merchant Name Vs Fraud Relation

In [53]:
merchant_counts = df['merchant_name'].value_counts()
merchant_counts.head()

merchant_name
Kilback LLC          58
Cormier LLC          48
Kutch and Sons       46
Rau and Sons         44
McDermott-Weimann    42
Name: count, dtype: int64

In [54]:
df_merchant = df[['merchant_name', 'is_fraud']].groupby(by='merchant_name').sum().reset_index().sort_values(by='is_fraud', ascending=False)
df_merchant['total'] = df_merchant.apply(lambda row: merchant_counts[row['merchant_name']], axis=1)
df_merchant = pd.concat([df_merchant.head(10), df_merchant.tail(10)], ignore_index=True)
print(df_merchant.shape)
df_merchant.head()

(20, 3)


,merchant_name,is_fraud,total
0,Kiehn-Emmerich,20,42
1,"""Romaguera, Cruickshank and Greenholt""",18,37
2,Kerluke-Abshire,18,35
3,Kunze Inc,16,31
4,Kilback LLC,15,58


In [55]:
# Initialize figure
fig = go.Figure()

# Add the first bar plot (total transactions) in light blue
fig.add_trace(go.Bar(
    x=df_merchant['merchant_name'], 
    y=df_merchant['total'], 
    name='Total Transactions',
    marker_color='lightblue'
))

# Percentage value
percent = (df_merchant['is_fraud']/df_merchant['total'])*100
txt = percent.apply(lambda x: f'{x: .1f}%')

# Add the second bar plot (fraud transactions) in darker blue, to be overlayed on the first
fig.add_trace(go.Bar(
    x=df_merchant['merchant_name'], 
    y=df_merchant['is_fraud'], 
    name='Fraud Transactions',
    marker_color='blue',
    text=txt,
    textposition='outside'  # Position the text labels outside the bars
))

# Update layout to overlay the bars
fig.update_layout(
    width=1000,
    height=400,
    barmode='overlay',  # Overlay the bars on top of each other
    title='Total vs. Fraudulent Transactions by Merchant Names (Top 10 & Bottom 10)',
    xaxis_title='Merchant Names',
    yaxis_title='Count of Transactions',
    xaxis=dict(
        tickmode='linear',  # Set x-ticks at regular intervals
        tick0=1,  # Starting tick value
        dtick=1,  # Interval between ticks
        range=[-1, len(df_merchant['merchant_name'])]  # Set x-axis limits (xlim)
    ),
    yaxis=dict(
#         range=[0, 150]  # Set y-axis limits (xlim)
    )   
)
fig.show()

Unlike the city coordinates the merchant coordinates are not unique - single merchant can have multiple locations - therefore in this case we will only consider the 'merchant_name' and drop the merchant coordinates

In [56]:
df.drop(columns=['merch_lat', 'merch_long'], inplace=True)
df.head(2)

,trans_hour,purchase_category,city_population,purchase_city_lat,purchase_city_long,amount,age,occupation,merchant_name,is_fraud
0,0,grocery_net,145,64.7556,-165.6723,14.37,80,"""Administrator, education""","""Stokes, Christiansen and Sipes""",1
1,15,shopping_net,145,64.7556,-165.6723,966.11,80,"""Administrator, education""",Predovic Inc,1


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14446 entries, 0 to 14445
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   trans_hour          14446 non-null  int32  
 1   purchase_category   14446 non-null  object 
 2   city_population     14446 non-null  int64  
 3   purchase_city_lat   14446 non-null  float64
 4   purchase_city_long  14446 non-null  float64
 5   amount              14446 non-null  float64
 6   age                 14446 non-null  int32  
 7   occupation          14446 non-null  object 
 8   merchant_name       14446 non-null  object 
 9   is_fraud            14446 non-null  int64  
dtypes: float64(3), int32(2), int64(2), object(3)
memory usage: 1015.9+ KB


In [58]:
cat_cols = ['purchase_category', 'occupation', 'merchant_name']
le = LabelEncoder()
label_encoder = {}
for col in cat_cols:
    df[col] = le.fit_transform(df[col])
    label_encoder[col] = le   

In [59]:
df.head(2)

,trans_hour,purchase_category,city_population,purchase_city_lat,purchase_city_long,amount,age,occupation,merchant_name,is_fraud
0,0,3,145,64.7556,-165.6723,14.37,80,1,179,1
1,15,11,145,64.7556,-165.6723,966.11,80,1,564,1


In [61]:
df.to_csv('credit_data_final.csv', index=False)